In [29]:
#!pip install opencv-python

In [30]:
import pandas as pd
import boto3
import json
import base64
from botocore.config import Config
import requests
import numpy as np
import io
from PIL import Image
import cv2
import pathlib
import boto3

agent_client = boto3.client('bedrock-agent-runtime',  region_name="ap-south-1")

In [31]:
p = pathlib.Path("data/tmp/maths/")
p.mkdir(parents=True, exist_ok=True)

In [32]:
config = Config(
    retries = dict(
        max_attempts = 20
    )
)

In [33]:
subject = 'maths'

data = pd.read_excel('Data-QnA-Academic.xlsx', sheet_name=subject)
#data = data.dropna()
data = data.rename(columns={'CORRECT ANSWER': 'Correct Answer'})
data['Correct Answer'] = data['Correct Answer'].apply(str)
data = data[~data['Correct Answer'].str.contains('?usp=', regex=False)]
data.head(2)

,mediaurl,subject,Correct Answer,Unnamed: 3
0,https://d2bps9p1kiy4ka.cloudfront.net/5eb393ee...,Mathematics,7,correct
1,https://d2bps9p1kiy4ka.cloudfront.net/5eb393ee...,Mathematics,2,correct


In [34]:
data.shape

(158, 4)

In [35]:
SYSTEM_PROMPT = ""

text_prompt = f"""You are an expert in solving difficult graduate level math problem {subject} question.
You will be given the question in form of an image, first extract and understand the images, 
some of images are hand written for math complex equation and be very precise in understanding the images and questions.
the result totatlly depand on the capabilitie of undertanding images
differentiation,log etc and you have to solve the question step by step and provide the correct answer.
you also have to return the original question and answer as it for referenace.
example - 
question - On selling a tea-set at 5% loss and a lemon-set at 15% gain, a crockery seller gains rupees 7.
If he sells the tea-set at 5% gain and the lemon-set at 10% gain, he gains rupees 13. 
Find the actual price of the tea-set and the lemon-set?.
Let's denote the cost price of the tea-set as \( T \) and the cost price of the lemon-set as \( L \).
Answer- 
### Formulate the equations
1. First Scenario:
   - Selling the tea-set at a 5% loss: Selling price of tea-set = \( T - 0.05T = 0.95T \)
   - Selling the lemon-set at a 15% gain: Selling price of lemon-set = \( L + 0.15L = 1.15L \)
   - Total gain = Rs 7:( (0.95T + 1.15L) - (T + L) = 7
   
   Simplifying the equation:
   0.95T + 1.15L - T - L = 7
   -0.05T + 0.15L = 7


2. Second Scenario:
   - Selling the tea-set at a 5% gain: Selling price of tea-set = \( T + 0.05T = 1.05T \)
   - Selling the lemon-set at a 10% gain: Selling price of lemon-set = \( L + 0.10L = 1.10L \)
   - Total gain = Rs 13:( (1.05T + 1.10L) - (T + L) = 13
   
   Simplifying the equation:
   1.05T + 1.10L - T - L = 13
 
   0.05T + 0.10L = 13 \quad \text

### Solve the system of equations
We have the system:
-0.05T + 0.15L = 7 \quad \text

0.05T + 0.10L = 13 \quad \text

First, add Equation 1 and Equation 2 to eliminate \( T \):
(-0.05T + 0.15L) + (0.05T + 0.10L) = 7 + 13

0.25L = 20

L = 80
Now substitute \( L = 80 \) back into Equation 2:
0.05T + 0.10 \times 80 = 13

0.05T + 8 = 13

0.05T = 5
T = 100

Conclusion
The cost price of the tea-set is Rs 100 and the cost price of the lemon-set is RS 80.

"""

question_prompt = """You will be given a question in form of an image, and you have to perform the following tasks - 
1. convert it into text form without missing any detail
2. assess whether complex mathematical calculation is required to solve the question, answer either "yes" or "no"
3. difficulty level of the question for a university student studying that subject as "low", "medium" or "high"

The output should be a python dictionary with following as the keys - "text", "calculation", and "difficulty".
Do not provide anything else in the output apart from the resulting python dictionary."""

In [36]:
# Initialize the Amazon Bedrock runtime client
client = boto3.client(service_name="bedrock-runtime", region_name="us-east-1")

# Invoke the model with the prompt and the encoded image
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
# model_id = "anthropic.claude-3-haiku-20240307-v1:0"

In [37]:
x  = f"write {question_prompt}"

In [38]:
x

'write You will be given a question in form of an image, and you have to perform the following tasks - \n1. convert it into text form without missing any detail\n2. assess whether complex mathematical calculation is required to solve the question, answer either "yes" or "no"\n3. difficulty level of the question for a university student studying that subject as "low", "medium" or "high"\n\nThe output should be a python dictionary with following as the keys - "text", "calculation", and "difficulty".\nDo not provide anything else in the output apart from the resulting python dictionary.'

In [39]:

from core_lib.math_question_generation import mathquestion

In [40]:
math = mathquestion()

### Inference

In [41]:
gt_list = []
ans_list = []
index_list = []
calc_list = []
diff_list = []
ques_list = []
cotanswer = []
#text_prompt = text_prompt.format(**{'subject': subject})
for i, r in data[0:10].iterrows():
    img_path = f'data/tmp/{subject}/{subject}_{i}.jpg'
    img_data = requests.get(r['mediaurl']).content
    image = np.array(Image.open(io.BytesIO(img_data)))
    if image.shape[0] > 1.3 * image.shape[1]:
        image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
        cv2.imwrite(img_path, image)
        with open(img_path, "rb") as image_file:
            image = base64.b64encode(image_file.read()).decode("utf8")
    else:
        with open(img_path, 'wb') as handler:
            handler.write(img_data)
        image = base64.b64encode(img_data).decode("utf8")
    res = math.call_claude_sonet(image, text_prompt)
    review_prompt = f"here is {res} question and answer of the image attached, your task is to evaluate the result as expert in mathematics like a graduate level expert, if the answer is not correct please change and modify as per image attached"
    res1 = math.call_claude_sonet(image, review_prompt)
    
    # ques = json.loads(invoke_claude_3_multimodal(question_prompt, image))
    # print(f"GT - {r['Correct Answer']}\n\n")
    # print(f"Answer - {res}")
    # print("Answeragent", agentres)
    gt_list.append(r['Correct Answer'])
    ans_list.append(res)
    cotanswer.append(res1)
    # ques_list.append(ques['text'])
    index_list.append(i)
    # calc_list.append(ques['calculation'])
    # diff_list.append(ques['difficulty'])
    print('*'*100)

****************************************************************************************************
****************************************************************************************************
****************************************************************************************************
****************************************************************************************************
****************************************************************************************************
****************************************************************************************************
****************************************************************************************************
****************************************************************************************************
****************************************************************************************************
*******************************************************************************************

In [42]:
df = pd.DataFrame({'Ind': index_list, 'GT': gt_list,
                   'Answer': ans_list, "Answer1":cotanswer})
df.to_csv(f'data/{subject}_answers_v3.csv', index=False)
